In [34]:
# To ignore warinings
import warnings
warnings.filterwarnings('ignore')

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [36]:
# Reading the dataset
df = pd.read_csv('/kaggle/input/nepali-news-raw/news.csv')

Before moving towards text cleaning, lets remove any null and duplicate values

In [37]:
df.drop_duplicates(["news"], inplace=True)
df.dropna(inplace=True, axis=0)

In [38]:
df.head()

,Unnamed: 0,title,news,category,published_date
0,0,एमालेको प्रदेश प्रतिनिधिमा उदयपुरबाट सर्वसम्मत,उदयपुर : नेकपा एमाले कोशी प्रदेश कमिटीको प्रथम...,समाज,"असोज ७, २०८० आइतबार १७:४३:४६"
1,1,गौशालाबाट लुटियो ६ लाख ५० हजार रुपैयाँ,महोत्तरी : गौशाला नगरपालिका-११ भरतपुरका शम्भु ...,समाज,"असोज ७, २०८० आइतबार १७:२३:५८"
2,2,"आगलागीमा ४ करोड बढीको क्षति, पीडितलाई तत्काल र...",मुगु : जिल्ला सदरमुकाम गमगढीमा शुक्रबार राति ...,समाज,"असोज ७, २०८० आइतबार १७:८:३९"
3,3,तलेजुको दर्शन गर्न पूर्वराजा ज्ञानेन्द्र शाह भ...,भक्तपुर : पूर्वराजा ज्ञानेन्द्र शाह भक्तपुर आए...,समाज,"असोज ७, २०८० आइतबार १६:५०:२१"
4,4,वीरगन्जको खेतबाट ८ हजार ९१५ पिस लागूऔषध बरामद,वीरगन्ज : वीरगन्ज महानगरपालिका-२२ मनियारीस्थि...,समाज,"असोज ७, २०८० आइतबार १६:४४:२५"


We will be using fasttext word embeddings which was trained on the same data for summarization and recommendation.

In [39]:
import fasttext

In [40]:
embeddings_model = fasttext.load_model("/kaggle/input/fasttext-embeddings-nepali-news/fasttext_summarizer_embeddings.bin")

In [41]:
embeddings_model.get_nearest_neighbors("बझाङ")

[(0.9865414500236511, 'बझाङमै'),
 (0.9853097796440125, 'बझाङकाे'),
 (0.9850979447364807, 'बाजुरा'),
 (0.9847890138626099, 'डोटी'),
 (0.9842208027839661, 'बझाङी'),
 (0.9822198748588562, 'बाजुङ'),
 (0.9821557402610779, 'बाजुराली'),
 (0.9816499948501587, 'बझाङकै'),
 (0.9804821014404297, 'बाजुराकी'),
 (0.979947566986084, 'बाजुराकै')]

In [42]:
import string
import codecs
import string
import re
from nltk.corpus import stopwords
import json
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
nep_stopwrods = stopwords.words("nepali")

In [44]:

def remove_emojis_english_and_numbers(data):
    """
    Removes emojis, non-nepali texts and numbers from the given text
    """
    # Removes emoji from given data
    emoj = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002500-\U00002BEF"  # chinese char
        "\U00002702-\U000027B0"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "\U0001f926-\U0001f937"
        "\U00010000-\U0010ffff"
        "\u2640-\u2642"
        "\u2600-\u2B55"
        "\u200d"
        "\u23cf"
        "\u23e9"
        "\u231a"
        "\ufe0f"  # dingbats
        "\u3030"
        "]+",
        re.UNICODE,
    )
    res = re.sub(emoj, "", data)
    res = re.sub("[0-9]+", "", res)
    return re.sub("[a-zA-Z]", "", res)


def preprocess_text(data):
    """
    Cleans the given textual data
    Removes special characters, english texts, numbers, and stopwords
    """
    if type(data) == float:
        return data
    data = (
        data.replace("-", " ")
        .replace("—", " ")
        .replace("‘", " ")
        .replace("’", " ")
        .replace("।", " ")
        .replace("–", " ")
        .replace("“", " ")
        .replace("”", " ")
        .replace("\n", " ")
        .replace("–", " ")
        .replace(" : ", ": ")
    )
    no_extra_spaces = " ".join(data.split())
    no_emoji_english_numbers = remove_emojis_english_and_numbers(no_extra_spaces)
    no_punc = "".join(
        [
            char
            for char in no_emoji_english_numbers
            if char not in list(string.punctuation)
        ]
    )
    extra = " ".join(no_punc.split())
    no_num = "".join([char for char in extra if char not in "०१२३४५६७८९"])
    no_stopwords = [
        word.strip() for word in no_num.split() if word.strip() not in nep_stopwrods
    ]
    return " ".join(no_stopwords)


Now lets clean the news articles

In [45]:
df["news_cleaned"] = df["news"].apply(preprocess_text)

For recommendation, we will only consider the first 300 words.<br>
So, I'll trim the news articles so that any article will have at most 300 words in it.<br>
Additionally, I will also remove all the news with less than 30 words.

In [46]:
CONSTANTS = {}
CONSTANTS["max_news_length"] = 300

In [47]:
# Compute length of each news articles
df["length"] = df["news_cleaned"].apply(lambda x: len(x.split()))
# Remove all news with <30 words
df = df[df["length"] >= 30]
# Remove all news with length > 300
df["news_cleaned"] = df["news_cleaned"].apply(lambda x: " ".join(x.split()[:CONSTANTS["max_news_length"]]))

Now, we will compute the average word vector over each of the news artcles.<br>
This way, each news articles will be represented by a vector of size 200.

In [48]:
def average_word_vector(text):
    word_vectors = []
    words = text.split()
    for word in words:
        word_vectors.append(embeddings_model.get_word_vector(word))
    return np.mean(np.array(word_vectors), axis=0)

In [68]:
df["news_embeddings"] = df["news_cleaned"].apply(average_word_vector)

In [69]:
df.head()

,Unnamed: 0,title,news,category,published_date,news_cleaned,length,news_embeddings
0,0,एमालेको प्रदेश प्रतिनिधिमा उदयपुरबाट सर्वसम्मत,उदयपुर : नेकपा एमाले कोशी प्रदेश कमिटीको प्रथम...,समाज,"असोज ७, २०८० आइतबार १७:४३:४६",उदयपुर नेकपा एमाले कोशी प्रदेश कमिटीको प्रथम अ...,275,"[-0.46222952, 0.4161641, -0.3515223, -0.363243..."
1,1,गौशालाबाट लुटियो ६ लाख ५० हजार रुपैयाँ,महोत्तरी : गौशाला नगरपालिका-११ भरतपुरका शम्भु ...,समाज,"असोज ७, २०८० आइतबार १७:२३:५८",महोत्तरी गौशाला नगरपालिका भरतपुरका शम्भु महतोक...,58,"[-0.4262865, 0.7805103, -0.3748274, -0.1953629..."
2,2,"आगलागीमा ४ करोड बढीको क्षति, पीडितलाई तत्काल र...",मुगु : जिल्ला सदरमुकाम गमगढीमा शुक्रबार राति ...,समाज,"असोज ७, २०८० आइतबार १७:८:३९",मुगु जिल्ला सदरमुकाम गमगढीमा शुक्रबार राति आगल...,168,"[-0.4308221, 0.75850004, -0.40295634, -0.42180..."
3,3,तलेजुको दर्शन गर्न पूर्वराजा ज्ञानेन्द्र शाह भ...,भक्तपुर : पूर्वराजा ज्ञानेन्द्र शाह भक्तपुर आए...,समाज,"असोज ७, २०८० आइतबार १६:५०:२१",भक्तपुर पूर्वराजा ज्ञानेन्द्र शाह भक्तपुर आएका...,52,"[-0.59828883, 0.409928, -0.2368881, -0.0784565..."
4,4,वीरगन्जको खेतबाट ८ हजार ९१५ पिस लागूऔषध बरामद,वीरगन्ज : वीरगन्ज महानगरपालिका-२२ मनियारीस्थि...,समाज,"असोज ७, २०८० आइतबार १६:४४:२५",वीरगन्ज वीरगन्ज महानगरपालिका मनियारीस्थित खेतम...,79,"[-0.36832175, 0.7596689, -0.38873523, -0.22425..."


In [71]:
df["news_embeddings"][0][:5]

array([-0.46222952,  0.4161641 , -0.3515223 , -0.36324397, -0.6637442 ],
      dtype=float32)

In [72]:
test_news = '''
काठमाडौँ — प्रधानमन्त्री पुष्पकमल दाहालले संघीय निजामती ऐन मन्त्रिपरिषद्‍बाट पारित भइसकेको जानकारी दिएका छन् । उनले उक्त ऐनलाई संसद्‍मा ल्याउने तयारी भइरहेको पनि जानकारी दिएका हुन् । 
राष्ट्रिय सभाको राष्ट्रिय सरोकार तथा समन्वय समितिको बुधबारको बैठकमा संघीय निजामती ऐन मन्त्रिपरिषद्‍बाट पारित भइसकेको जानकारी दिएका हुन् । 'पछिल्लो मन्त्रिपरिषद् बैठकले संघीय निजामती ऐनलाई टुंग्याएर संसद्माा दर्ता गर्ने तयारी भइरहेको छ,' उनले भने ।
संघ, प्रदेश र स्थानीय तहमा प्रशासनिक संघीयता कार्यान्वयनका लागि संघीय निजामती ऐन सरकारले अगाडि बढाउन लागेको हो । प्रधानमन्त्री दाहालले संघ, प्रदेश र स्थानीय तहबीच पछिल्लो समयमा निरन्तर सहकार्य भइरहेको बताएका छन् । उनले तीन तहबीच कार्यक्षेत्र र साझा अधिकारका विषयमा छलफलबाटै अगाडि बढिरहेको बताए । तीन तहबीच अधिकारसूचीको कार्यविस्तृतिकरण गर्न लागिएको दाहालले जानकारी दिए ।

प्रधानमन्त्री दाहालले आफ्नो नेतृत्वको सरकार गठन भएदेखि नै सुशासन, समृद्धि र सामाजिक न्यायलाई प्राथमिकता दिएर अगाडि बढिरहेको बताए ।
'''

In [74]:
# Remove the new line escape sequence
news = test_news.replace("\n", " ")

# Remove unnecessary whitespaces from the news
news = " ".join(news.split())

In [75]:
# Clean the input news
cleaned_news = preprocess_text(news)

Since, we will only consider the 1st 300 words for recommendation, we will trim away excess words from the cleaned news if any

In [78]:
if len(cleaned_news.split()) > CONSTANTS["max_news_length"]:
    cleaned_news = " ".join(cleaned_news.split()[: CONSTANTS["max_news_length"]])

Now we compute the average word vector for the test news

In [79]:
test_news_embeddings = average_word_vector(cleaned_news)

In [80]:
test_news_embeddings

array([-0.43605602,  0.3846968 , -0.10406514, -0.5063388 , -0.6339863 ,
        0.09585791, -0.43718696,  0.31419364,  0.09286163,  0.28045174,
       -0.3276915 ,  0.29064444, -0.684737  , -0.39856577, -0.54183024,
        0.07036943,  0.35736802, -0.20867892,  0.40017733, -0.23223512,
        0.28835994,  0.05861878, -0.3617989 , -0.27294636,  0.05947028,
        0.05400606, -0.4161799 ,  0.2019734 , -0.25376743,  0.13135552,
        0.52459425, -0.1985665 , -0.04029851,  0.5810648 , -0.13340227,
        0.13140322,  0.6447469 ,  0.3142906 , -0.64475954, -0.35093156,
       -0.5150421 ,  0.45044962,  0.15904687,  0.2537691 , -0.10700988,
       -0.11643191, -0.06669387,  0.08491957, -0.2123762 , -0.4124716 ,
       -0.20621754, -0.47859603, -0.63725394, -0.17214352, -0.3273948 ,
       -0.8860045 ,  0.46025   ,  0.10332796,  0.37505478,  0.02759538,
        0.5159431 ,  0.80247307, -0.21577412,  0.4243612 ,  0.24061465,
        0.16935192,  0.05311522,  0.32252645,  0.5025418 , -0.85

We now compute the cosine similarity between the test news embeddings and all the news

In [97]:
cosine_similarity([df["news_embeddings"][0]], [test_news_embeddings]).shape

(1, 1)

In [98]:
cosine_similarities = [(news, cosine_similarity([embedding], [test_news_embeddings])[0][0]) for news, embedding in zip(df["news"], df["news_embeddings"])]

In [99]:
cosine_similarities.sort(reverse=True, key=lambda k: k[1])

In [100]:
cosine_similarities[:5]

[('सङ्घीय मामिला तथा सामान्य प्रशासन मन्त्रालयले सङ्घीय निजामती ऐनलाई अन्तिम टुङ्गो लगाउने कामलाई तीव्रता दिएको छ  ।  संसद्को जारी अधिवेशनमा संघीय निजामती ऐनसम्बन्धी विधेयक संसदमा पेश गर्ने तयारी भइरहेका बेला विज्ञहरुले सो सुझाव दिनुभएको हो  ।   सामान्य प्रशासनमन्त्री अमनलाल मोदीले आज गर्नुभएको छलफलमा विज्ञहरुले त्यस्तो सुझाव दिनुभएको छ  ।  मन्त्रालयले आज सङ्घीय मामिला र निजामती कर्मचारी प्रशासनका विज्ञसँग बबरमहलस्थित पीएलजीएसपीको कार्यालयमा पूर्वसचिवसँग परामर्श गरेको थियो  ।    छलफलमा पूर्वसचिव मधुरमण आचार्य, डा. मानबहादुर विके, विमल वाग्ले, बालकृष्ण प्रसाई, कृष्णहरि बास्कोटा र शङ्कर अधिकारी सहभागी हुनुहुन्थ्यो  ।  पूर्वसचिवहरुले संघीयता कार्यान्व्यनका लागि निर्णायक मानिएको संघीय निजामती ऐन कार्यान्वयनमा ल्याउन ढिला भएको उल्लेख गर्दै समयसापेक्ष बनाएर अघि बढाउन सुझावहरु दिएको मन्त्री मोदीको सचिवालयले जनाएको छ  ।   नेपालमा सङ्घीयता लागू भए पनि सङ्घीय निजामती ऐन, संघीय प्रहरी ऐनजस्ता कानुन नबन्दा सङ्घीयता कार्यान्वयनमा चुनौती देखा पर्नेगरेको छ  ।  सोही संवेदनशीलतालाई मध्यनजर गर्दै मन्त्र

Now lets save the news articles along with their associated embeddings

In [101]:
df.head()

,Unnamed: 0,title,news,category,published_date,news_cleaned,length,news_embeddings
0,0,एमालेको प्रदेश प्रतिनिधिमा उदयपुरबाट सर्वसम्मत,उदयपुर : नेकपा एमाले कोशी प्रदेश कमिटीको प्रथम...,समाज,"असोज ७, २०८० आइतबार १७:४३:४६",उदयपुर नेकपा एमाले कोशी प्रदेश कमिटीको प्रथम अ...,275,"[-0.46222952, 0.4161641, -0.3515223, -0.363243..."
1,1,गौशालाबाट लुटियो ६ लाख ५० हजार रुपैयाँ,महोत्तरी : गौशाला नगरपालिका-११ भरतपुरका शम्भु ...,समाज,"असोज ७, २०८० आइतबार १७:२३:५८",महोत्तरी गौशाला नगरपालिका भरतपुरका शम्भु महतोक...,58,"[-0.4262865, 0.7805103, -0.3748274, -0.1953629..."
2,2,"आगलागीमा ४ करोड बढीको क्षति, पीडितलाई तत्काल र...",मुगु : जिल्ला सदरमुकाम गमगढीमा शुक्रबार राति ...,समाज,"असोज ७, २०८० आइतबार १७:८:३९",मुगु जिल्ला सदरमुकाम गमगढीमा शुक्रबार राति आगल...,168,"[-0.4308221, 0.75850004, -0.40295634, -0.42180..."
3,3,तलेजुको दर्शन गर्न पूर्वराजा ज्ञानेन्द्र शाह भ...,भक्तपुर : पूर्वराजा ज्ञानेन्द्र शाह भक्तपुर आए...,समाज,"असोज ७, २०८० आइतबार १६:५०:२१",भक्तपुर पूर्वराजा ज्ञानेन्द्र शाह भक्तपुर आएका...,52,"[-0.59828883, 0.409928, -0.2368881, -0.0784565..."
4,4,वीरगन्जको खेतबाट ८ हजार ९१५ पिस लागूऔषध बरामद,वीरगन्ज : वीरगन्ज महानगरपालिका-२२ मनियारीस्थि...,समाज,"असोज ७, २०८० आइतबार १६:४४:२५",वीरगन्ज वीरगन्ज महानगरपालिका मनियारीस्थित खेतम...,79,"[-0.36832175, 0.7596689, -0.38873523, -0.22425..."


In [102]:
df["Unnamed: 0"].head()

0    0
1    1
2    2
3    3
4    4
Name: Unnamed: 0, dtype: int64

In [103]:
df.drop(columns=["Unnamed: 0", "published_date", "news_cleaned", "length"], inplace=True)

In [104]:
df.head()

,title,news,category,news_embeddings
0,एमालेको प्रदेश प्रतिनिधिमा उदयपुरबाट सर्वसम्मत,उदयपुर : नेकपा एमाले कोशी प्रदेश कमिटीको प्रथम...,समाज,"[-0.46222952, 0.4161641, -0.3515223, -0.363243..."
1,गौशालाबाट लुटियो ६ लाख ५० हजार रुपैयाँ,महोत्तरी : गौशाला नगरपालिका-११ भरतपुरका शम्भु ...,समाज,"[-0.4262865, 0.7805103, -0.3748274, -0.1953629..."
2,"आगलागीमा ४ करोड बढीको क्षति, पीडितलाई तत्काल र...",मुगु : जिल्ला सदरमुकाम गमगढीमा शुक्रबार राति ...,समाज,"[-0.4308221, 0.75850004, -0.40295634, -0.42180..."
3,तलेजुको दर्शन गर्न पूर्वराजा ज्ञानेन्द्र शाह भ...,भक्तपुर : पूर्वराजा ज्ञानेन्द्र शाह भक्तपुर आए...,समाज,"[-0.59828883, 0.409928, -0.2368881, -0.0784565..."
4,वीरगन्जको खेतबाट ८ हजार ९१५ पिस लागूऔषध बरामद,वीरगन्ज : वीरगन्ज महानगरपालिका-२२ मनियारीस्थि...,समाज,"[-0.36832175, 0.7596689, -0.38873523, -0.22425..."


In [105]:
df.to_csv("/kaggle/working/news_raw_with_embeddings", index=None)

In [113]:
df_new = pd.read_csv("/kaggle/working/news_raw_with_embeddings")

In [114]:
df_new.head()

,title,news,category,news_embeddings
0,एमालेको प्रदेश प्रतिनिधिमा उदयपुरबाट सर्वसम्मत,उदयपुर : नेकपा एमाले कोशी प्रदेश कमिटीको प्रथम...,समाज,[-0.46222952 0.4161641 -0.3515223 -0.363243...
1,गौशालाबाट लुटियो ६ लाख ५० हजार रुपैयाँ,महोत्तरी : गौशाला नगरपालिका-११ भरतपुरका शम्भु ...,समाज,[-4.26286489e-01 7.80510306e-01 -3.74827415e-...
2,"आगलागीमा ४ करोड बढीको क्षति, पीडितलाई तत्काल र...",मुगु : जिल्ला सदरमुकाम गमगढीमा शुक्रबार राति ...,समाज,[-0.4308221 0.75850004 -0.40295634 -0.421803...
3,तलेजुको दर्शन गर्न पूर्वराजा ज्ञानेन्द्र शाह भ...,भक्तपुर : पूर्वराजा ज्ञानेन्द्र शाह भक्तपुर आए...,समाज,[-0.59828883 0.409928 -0.2368881 -0.078456...
4,वीरगन्जको खेतबाट ८ हजार ९१५ पिस लागूऔषध बरामद,वीरगन्ज : वीरगन्ज महानगरपालिका-२२ मनियारीस्थि...,समाज,[-0.36832175 0.7596689 -0.38873523 -0.224259...


In [115]:
df_new["news_embeddings"][0]

'[-0.46222952  0.4161641  -0.3515223  -0.36324397 -0.6637442  -0.02297877\n -0.30172056  0.16980508 -0.04536987  0.20581713 -0.4536083   0.22299013\n -0.6308252  -0.30797336 -0.36296865  0.01073156  0.05223433 -0.24868985\n  0.48315573 -0.34785149  0.34125528  0.01687056 -0.23585446 -0.32480276\n  0.00797121 -0.01973209 -0.23638684  0.26384318 -0.2273136   0.2648312\n  0.3386087  -0.27324346 -0.02599072  0.72693217 -0.19403026  0.00178169\n  0.7401632   0.27499986 -0.58513266 -0.3072183  -0.2582888   0.4111524\n  0.04413545  0.09004416 -0.09735692 -0.08581097 -0.00498853  0.1416967\n -0.30987892 -0.25629818 -0.13254763 -0.56647646 -0.4224284  -0.27967006\n -0.23232421 -0.7469518   0.49742442  0.22415355  0.32527158 -0.08479398\n  0.48176447  0.8532954  -0.25825545  0.4405628   0.15466596  0.3408158\n  0.00669588  0.15779662  0.36192635 -0.7273691  -0.2564263  -0.08688225\n  0.48723432 -0.30415323 -0.13224943  0.15719499  0.14244418 -0.07470571\n  0.1505239  -0.32876402  0.2588405  -0.2

In [127]:
read_text = df_new["news_embeddings"][0]
read_text = read_text.replace("\n", "").replace("[", "").replace("]", "")
# read_text = " ".join(read_text.split())
np.fromstring(read_text,sep=' ', dtype=float)

array([-0.46222952,  0.4161641 , -0.3515223 , -0.36324397, -0.6637442 ,
       -0.02297877, -0.30172056,  0.16980508, -0.04536987,  0.20581713,
       -0.4536083 ,  0.22299013, -0.6308252 , -0.30797336, -0.36296865,
        0.01073156,  0.05223433, -0.24868985,  0.48315573, -0.34785149,
        0.34125528,  0.01687056, -0.23585446, -0.32480276,  0.00797121,
       -0.01973209, -0.23638684,  0.26384318, -0.2273136 ,  0.2648312 ,
        0.3386087 , -0.27324346, -0.02599072,  0.72693217, -0.19403026,
        0.00178169,  0.7401632 ,  0.27499986, -0.58513266, -0.3072183 ,
       -0.2582888 ,  0.4111524 ,  0.04413545,  0.09004416, -0.09735692,
       -0.08581097, -0.00498853,  0.1416967 , -0.30987892, -0.25629818,
       -0.13254763, -0.56647646, -0.4224284 , -0.27967006, -0.23232421,
       -0.7469518 ,  0.49742442,  0.22415355,  0.32527158, -0.08479398,
        0.48176447,  0.8532954 , -0.25825545,  0.4405628 ,  0.15466596,
        0.3408158 ,  0.00669588,  0.15779662,  0.36192635, -0.72

In [128]:
def makeArray(text):
    text = text.replace("\n", "").replace("[", "").replace("]", "")
    return np.fromstring(text,sep=' ')

df_new['news_embeddings'] = df_new['news_embeddings'].apply(makeArray)

In [130]:
df_new["news_embeddings"][0].shape

(200,)